In [63]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
from tqdm.notebook import tqdm
from itertools import product
from math import ceil, log
import pennylane as qml
import numpy as np
import torch

In [56]:
class Dataset:
    def __init__(self, features, labels, batch_size):
        # Randomly shuffle indices of the data
        idx = np.random.permutation(len(features))
        
        # Split data into batches
        num_splits = ceil(len(features)/batch_size)
        self.features = Variable(torch.tensor(np.split(features[idx], num_splits)), requires_grad=False)
        self.labels = Variable(torch.tensor(np.split(labels[idx], num_splits)), requires_grad=False)
        
        self.batch_size = batch_size
        self.counter = 0
        assert len(self.features) == len(self.labels), "features and labels have different sizes"
        
    def sample_batch(self):
        """
        A method that samples a batch from the dataset
        """
        features = self.features[self.counter]
        labels = self.labels[self.counter]
        self.counter = (self.counter + 1) % len(self.features)
        return features, labels
    
    def __len__(self):
        return len(self.features)
    
def normalize (vector):
    """
    Normalizes vector squared amplitudes to one
    """
    norm = np.sqrt(np.sum(vector ** 2, -1))
    r = []
    for i in range(len(vector)):
        r.append(vector[i,:]/norm[i])
    r = np.array(r)
    return r

In [57]:
# Define number of datapoints and samples to extract from distribution
datapoints, samples = 100000, 64

# Generating class 1 data
mean, std = -20, 1
data1 = normalize(std * np.random.randn(datapoints//2, samples) + mean)
labels1 = np.zeros(datapoints//2)

# Generating class 2 data
mean, std = 5, 3
data2 = normalize(std * np.random.randn(datapoints//2, samples) + mean)
labels2 = np.ones(datapoints//2)

# Concatenating data
data = np.concatenate((data1, data2))
labels = np.concatenate((labels1, labels2))

# Splitting data into train, test and validation
test_split, val_split, random_state = 0.2, 0.3, 42
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=test_split, random_state=random_state)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_split, random_state=random_state)

# Creating the datasets for train, test and validation
batch_size = 32
trainset = Dataset(X_train, y_train, 32)
valset = Dataset(X_val, y_val, 32)
testset = Dataset(X_test, y_test, 32)

In [6]:
def combinations(values):
    """
    Calculates every possible pair combination of values
    """
    r = []
    for i in range(len(values)):
        for j in range(i+1,len(values)):
            r.append((i,j))
    return r

In [38]:
# number of qubits in the circuit
nr_qubits = ceil(log(samples,2))
# number of layers in the circuit
nr_layers = 2

# randomly initialize parameters from a normal distribution
params = np.random.normal(0, np.pi, (nr_layers, nr_qubits, 3))
params = Variable(torch.tensor(params), requires_grad=True)

# a layer of the circuit ansatz
def layer(params, j):
    for i in range(params.shape[1]):
        qml.RX(params[j, i, 0], wires=i)
        qml.RY(params[j, i, 1], wires=i)
        qml.RZ(params[j, i, 2], wires=i)

    # Add CNOT for every qubit combination
    combs = combinations(range(params.shape[1]))
    for i, k in combs:
        qml.CNOT(wires=[i, k])

In [44]:
dev = qml.device("default.qubit", wires=nr_qubits)

In [66]:
@qml.qnode(dev, interface="torch")
def circuit(params, x):
    # Encoding
    qml.QubitStateVector(x, wires=range(params.shape[1]))
    # repeatedly apply each layer in the circuit
    for j in range(params.shape[0]):
        layer(params, j)

    # returns the expectation of the input matrix A on the first qubit
    return qml.expval(qml.PauliZ(0))

def qvc(params, features):
    predictions = [circuit(params, x) for x in features]
    return predictions

# cost function
def loss_fn(predictions, labels):
    """
    A simple square loss function to evaluate predictions
    """
    loss = 0.0
    for p, l in zip(predictions, labels):
        loss += (l - p) ** 2
    return loss

# set up the optimizer
optimizer = torch.optim.Adam([params], lr=0.1)

In [68]:
def get_classes(y_hat):
    r = []
    for y in y_hat:
        if y > 0.5:
            r.append(1)
        else:
            r.append(0)
    return r

In [71]:
# Number of epochs to train on
epochs = 10

# Performing the training loop
epoch_tqdm = tqdm(range(epochs), total=epochs, desc="Fitting")
for epoch in epoch_tqdm:
    # Iterate training data
    tr_accs, tr_losses = [], []
    tr_tqdm = tqdm(range(len(trainset)), total=len(trainset), desc="Training")
    for _ in tr_tqdm:
        # Sample batch of data and get predictions
        X, y = trainset.sample_batch()
        y_hat = qvc(params, X)

        # Get loss and update parameters
        optimizer.zero_grad()
        loss = loss_fn(y_hat, y)
        loss.backward()
        optimizer.step()
        
        # Add training info to tqdm progress bar
        preds = get_classes(y_hat)
        tr_acc = accuracy_score(y, preds)
        info = {"loss": loss.item(), "acc": tr_acc}
        tr_tqdm.set_postfix(info)
        tr_accs.append(tr_acc)
        tr_losses.append(loss.item())
        
    # Calculate training epoch accuracy and loss
    tr_acc = sum(tr_accs) / len(tr_accs)
    tr_loss = sum(tr_losses) / len(tr_losses)
        
    # Iterate validation data
    val_accs, val_losses = [], []
    val_tqdm = tqdm(range(len(trainset)), total=len(trainset), desc="Validating")
    with torch.no_grad():
        for _ in val_tqdm:
            # Sample batch of data and get predictions
            X, y = valset.sample_batch()
            y_hat = qvc(params, X)
            loss = loss_fc(y, y_hat)

            # Add validation info to tqdm progress bar
            preds = get_classes(y_hat)
            val_acc = accuracy_score(y, preds)
            info = {"loss": loss.item(), "acc": val_acc}
            val_tqdm.set_postfix(info)
            val_accs.append(val_acc)
            val_losses.append(loss.item())
            
    # Calculate training epoch accuracy and loss
    val_acc = sum(val_accs) / len(val_accs)
    val_loss = sum(val_losses) / len(val_losses)
    
    # Update info to tqdm bar
    info = {"tr_acc": tr_acc, "tr_loss": tr_loss, "val_acc": val_acc, "val_loss": val_loss}
    epoch_tqdm.set_postfix(info)

RuntimeError: 